In [1]:

import sys
import os

# Get the absolute path of the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

# Add the project root directory to the Python path
sys.path.append(project_root)
# sys.path.append(os.getcwd())

In [2]:
import pandas as pd

project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

# Define the path to the data directory
data_dir = "C:/Users/samkr/OneDrive/Desktop/AutoML/data" 


In [3]:
# Example: Access a specific data file in the data directory

df1 = pd.read_csv(os.path.join(data_dir, 'aim2reduce/X_train.csv'), index_col=0)
df2 = pd.read_csv(os.path.join(data_dir, 'aim2reduce/Y_train.csv'), index_col=0)
 
data = pd.merge(df1, df2, left_index=True, right_index=True)
 
from AutoML.analyzer import AutoMLAnalyzer
 
analyzer = AutoMLAnalyzer(data, target_variable='ED_visit', output_dir='./analysis_aim2reduce')
analyzer.run()


Config file not found, used a heuristic to define categorical and continuous columns. Please review!
Categorical: ['cancer_site_C17', 'regimen_GI_CISPCAPE_TRAS_MAIN_', 'esas_tiredness_is_missing', 'cancer_site_C18', 'esas_appetite_change_is_missing', 'cancer_site_C13', 'regimen_GI_MITOFU', 'cancer_site_C06', 'red_blood_cell_change_is_missing', 'cancer_site_C11', 'neutrophil_is_missing', 'cancer_site_C03', 'cancer_site_C12', 'red_blood_cell_is_missing', 'white_blood_cell_change_is_missing', 'lymphocyte_change_is_missing', 'regimen_GI_FLOT _GASTRIC_', 'cancer_site_C31', 'platelet_change_is_missing', 'intent_NEOADJUVANT', 'esas_well_being_is_missing', 'phosphate_change_is_missing', 'regimen_GI_ECF', 'regimen_GI_GEMCISP _BILIARY_', 'bicarbonate_is_missing', 'female', 'regimen_GI_FOLFNALIRI _COMP_', 'esas_depression_change_is_missing', 'regimen_GI_RAMUC_CCO__PACL_W_', 'cancer_site_C22', 'esas_anxiety_change_is_missing', 'cancer_site_C24', 'regimen_GI_PACLI_CARBO WEEKX5', 'regimen_GI_PANITUM

C:\Users\samkr\OneDrive\Desktop\AutoML\.pixi\envs\default\lib\site-packages\pandas\core\indexes\multi.py:643: DeprecationWarning: `cumproduct` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `cumprod` instead.
  codes = cartesian_product(codes)
C:\Users\samkr\OneDrive\Desktop\AutoML\.pixi\envs\default\lib\site-packages\pandas\core\reshape\util.py:60: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  return [
C:\Users\samkr\OneDrive\Desktop\AutoML\.pixi\envs\default\lib\site-packages\pandas\core\reshape\util.py:60: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  return [


╒════════════════════════════════════════════════════════════════════╤═════════════════════╤═══════════╤════════════════╕
│                                                                    │                     │ Missing   │ Overall        │
╞════════════════════════════════════════════════════════════════════╪═════════════════════╪═══════════╪════════════════╡
│ n                                                                  │                     │           │ 50             │
├────────────────────────────────────────────────────────────────────┼─────────────────────┼───────────┼────────────────┤
│ visit_month_sin, mean (SD)                                         │                     │ 0         │ 0.4 (1.0)      │
├────────────────────────────────────────────────────────────────────┼─────────────────────┼───────────┼────────────────┤
│ mean_corpuscular_volume, mean (SD)                                 │                     │ 0         │ -1.7 (2.4)     │
├───────────────────────

C:\Users\samkr\OneDrive\Desktop\AutoML\.pixi\envs\default\lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])
C:\Users\samkr\OneDrive\Desktop\AutoML\.pixi\envs\default\lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])
C:\Users\samkr\OneDrive\Desktop\AutoML\.pixi\envs\default\lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.pr